In [49]:
%matplotlib notebook

import sys
import os
import time
from importlib import reload
import time

In [2]:
import numpy as np
import qcodes as qc
import broadbean as bb

from matplotlib import pyplot as plt
from broadbean.plotting import plotter

In [3]:
from qcodes.dataset.measurements import Measurement

from pytopo.qctools import instruments as instools; reload(instools)
from pytopo.qctools.instruments import create_inst, add2station

from pytopo.qctools import measurement; reload(measurement)
from pytopo.qctools.measurement import MeasurementExt

from pytopo.mplplots.init_nb_plotting import *
from pytopo.mplplots import plots as mplplots
from pytopo.mplplots import tools as mpltools

import plottr; reload(plottr)

from pytopo.qctools.dataset2 import select_experiment
from plottr.qcodes_dataset import QcodesDatasetSubscriber

# Acquisition controllers

In [122]:
from qcodes.instrument_drivers.AlazarTech.ATS import AcquisitionController

class BaseAcqCtl(AcquisitionController):

    ZERO = np.int16(2048)
    RANGE = 2047.5
    MINSAMPLES = 384
    DATADTYPE = np.uint16

    def __init__(self, name, alazar_name, **kwargs):
        self.acquisitionkwargs = {}
        self.number_of_channels = 2
        self.trigger_func = None
        
        self._average_buffers = False

        try:
            super().__init__(name, alazar_name, **kwargs)
        except KeyError:
            raise
            self._alazar = None
        
        if self._alazar is not None:
            alz = self._get_alazar()
            self.add_parameter('sample_rate', get_cmd=alz.sample_rate)
            self.add_parameter('samples_per_record', get_cmd=alz.samples_per_record)
            self.add_parameter('records_per_buffer', get_cmd=alz.records_per_buffer)
            self.add_parameter('buffers_per_acquisition', get_cmd=alz.buffers_per_acquisition)

            self.add_parameter('acq_time', get_cmd=None, set_cmd=None, unit='s', initial_value=None)
            self.add_parameter("acquisition", get_cmd=self.do_acquisition, snapshot_value=False)
        else:
            self.add_parameter('sample_rate', set_cmd=None)
            self.add_parameter('samples_per_record', set_cmd=None)
            self.add_parameter('records_per_buffer', set_cmd=None)
            self.add_parameter('buffers_per_acquisition', set_cmd=None)


    # Functions that need to be implemented by child classes
    def data_shape(self):
        raise NotImplementedError

    def data_dims(self):
        raise NotImplementedError

    def process_buffer(self, buf):
        raise NotImplementedError

    def time2samples(self, t):
        nsamples_ideal = t * self.sample_rate()
        nsamples = int(nsamples_ideal // 128 * 128)
        if nsamples / self.sample_rate() < t:
            nsamples += 128
        return max(self.MINSAMPLES, nsamples)

    def pre_start_capture(self):
        self.tvals = np.arange(self.samples_per_record(), dtype=np.float32) / self.sample_rate()
        self.buffer_shape = (self.records_per_buffer(),
                             self.samples_per_record(),
                             self.number_of_channels)

        self.data = np.zeros(self.data_shape(), dtype=self.DATADTYPE)
        self.handling_times = np.zeros(self.buffers_per_acquisition(), dtype=np.float64)

    def pre_acquire(self):
        pass

    def handle_buffer(self, data, buffer_number=None):
        t0 = time.perf_counter()
        data.shape = self.buffer_shape
        data.dtype = np.uint16
        
        if not buffer_number or self._average_buffers:
            self.data += self.process_buffer(data)
            self.handling_times[0] = (time.perf_counter() - t0) * 1e3
        else:
            self.data[buffer_number] = self.process_buffer(data)
            self.handling_times[buffer_number] = (time.perf_counter() - t0) * 1e3

    def update_acquisitionkwargs(self, **kwargs):
        if self.acq_time() and 'samples_per_record' not in kwargs:
            kwargs['samples_per_record'] = self.time2samples(self.acq_time())
        self.acquisitionkwargs.update(**kwargs)
    
    def do_acquisition(self):
        if self._alazar is not None:
            value = self._get_alazar().acquire(acquisition_controller=self, **self.acquisitionkwargs)
        else:
            value = None
        return value


class RawAcqCtl(BaseAcqCtl):

    def data_shape(self):
        shp = (self.buffers_per_acquisition(),
               self.records_per_buffer(),
               self.samples_per_record(),
               self.number_of_channels)
        
        if not self._average_buffers:
            return shp
        else:
            return shp[1:]

    def data_dims(self):
        dims = ('buffers', 'records', 'samples', 'channels')
        
        if not self._average_buffers:
            return dims
        else:
            return dims[1:]

    def process_buffer(self, buf):
        return buf
    
    def post_acquire(self):
        return (np.right_shift(self.data, 4).astype(np.float32) - 2048) / 4096
    

class AvgBufCtl(BaseAcqCtl):
    
    DATADTYPE = np.uint32 # float32
    
    def __init__(self, *arg, **kw):
        super().__init__(*arg, **kw)    
        
        self._average_buffers = True
    
    def data_shape(self):
        shp = (self.records_per_buffer(),
               self.samples_per_record(),
               self.number_of_channels)
        return shp

    def data_dims(self):
        dims = ('records', 'samples', 'channels')
        return dims

    def process_buffer(self, buf):
        return buf # (np.right_shift(buf, 4).astype(np.float32) - 2048) / 4096
    
    def post_acquire(self):
        return (np.right_shift(self.data, 4).astype(np.float32) / self.buffers_per_acquisition() - 2048) / 4096


# Instantiate instruments

In [123]:
inst_list = []

# from pytopo.rf import alazar_acquisition; reload(alazar_acquisition)
# from pytopo.rf.alazar_acquisition import RawAcqCtl, DemodAcqCtl, DemodRelAcqCtl, IQRelAcqCtl, IQAcqCtl

from qcodes.instrument_drivers.AlazarTech.ATS9360 import AlazarTech_ATS9360
alazar = instools.create_inst(AlazarTech_ATS9360, 'alazar')
inst_list.append(alazar)

rawctl = create_inst(RawAcqCtl, 'rawctl', 'alazar', force_new_instance=True)
avgctl = create_inst(AvgBufCtl, 'avgctl', 'alazar', force_new_instance=True)
inst_list.append(rawctl)
inst_list.append(avgctl)

# demod_acq = instools.create_inst(DemodAcqCtl, 'demod_acq', 'alazar', force_new_instance=True)
# inst_list.append(demod_acq)

# demodrel_acq = instools.create_inst(DemodRelAcqCtl, 'rel_acq', 'alazar', force_new_instance=True)
# inst_list.append(demodrel_acq)

# iqrel_acq = instools.create_inst(IQRelAcqCtl, 'iqrel_acq', 'alazar', force_new_instance=True)
# inst_list.append(iqrel_acq)

# iq_acq = instools.create_inst(IQAcqCtl, 'iq_acq', 'alazar', force_new_instance=True)
# inst_list.append(iq_acq)

from qcodes.instrument_drivers.tektronix.AWG5014 import Tektronix_AWG5014
awg = instools.create_inst(Tektronix_AWG5014, 'awg', address="TCPIP0::169.254.220.147::inst0::INSTR")
inst_list.append(awg)

from qcodes.instrument_drivers.rohde_schwarz.SGS100A import RohdeSchwarz_SGS100A
LO = instools.create_inst(RohdeSchwarz_SGS100A, 'LO', address="TCPIP0::169.254.2.20")
inst_list.append(LO)
RF = instools.create_inst(RohdeSchwarz_SGS100A, 'RF', address="TCPIP0::169.254.231.38")
inst_list.append(RF)

from qcodes.instrument_drivers.Minicircuits.RUDAT_13G_90 import RUDAT_13G_90_USB
ro_attenuator = instools.create_inst(RUDAT_13G_90_USB, 'ro_attenuator')
inst_list.append(ro_attenuator)

station = qc.Station(*inst_list)

Instrument alazar already exists.
Instrument rawctl already exists.
Instrument avgctl already exists.
Instrument awg already exists.
Instrument LO already exists.
Instrument RF already exists.
Instrument ro_attenuator already exists.


Snapshot: Could not update parameter:buffer_timeout


# Make a test AWG sequence

In [91]:
### params

pulse_len = 8e-6
seq_len = 100e-6

### construct sequence
ramp = bb.PulseAtoms.ramp

ro = bb.BluePrint()
ro.setSR(1e9)

ro.insertSegment(0, ramp, (0, 0), dur=seq_len, name='ro_segment')
ro.marker1 = [(1e-6, 100e-9)]
ro.marker2 = [(2e-6, pulse_len)]

elem = bb.Element()
elem.addBluePrint(1, ro)

seq = bb.Sequence()
seq.name = 'test'
seq.addElement(1, elem)
seq.setSR(1e9)

seq.setChannelAmplitude(1, 1)
seq.setChannelOffset(1, 0)

seq.setSequencingTriggerWait(1, 0)
seq.setSequencingGoto(1, 1)

pkg = seq.outputForAWGFile()

awg.clock_freq(1e9)
awg.make_send_and_load_awg_file(*pkg[:])
awg.set('ch1_state', 1)
awg.start()

'Running'

# Configure Alazar and run

In [143]:
IF = 20e6
acq_time = 0.1
ro_attenuator.attenuation(0)

### ATS settings 

ats_settings = dict(
    clock_source='INTERNAL_CLOCK',
    sample_rate=int(1e8),
    clock_edge='CLOCK_EDGE_RISING',
    decimation=1,
    coupling=['DC','DC'],
    channel_range=[.4, .4],
    impedance=[50, 50],
    trigger_operation='TRIG_ENGINE_OP_J',
    trigger_engine1='TRIG_ENGINE_J',
    trigger_source1='EXTERNAL',
    trigger_slope1='TRIG_SLOPE_POSITIVE',
    trigger_level1=128+20,
    trigger_engine2='TRIG_ENGINE_K',
    trigger_source2='DISABLE',
    trigger_slope2='TRIG_SLOPE_POSITIVE',
    trigger_level2=128+5,
    external_trigger_coupling='DC',
    external_trigger_range='ETR_2V5',
    trigger_delay=0,
    timeout_ticks=int(1e7),
    aux_io_mode='AUX_IN_AUXILIARY',
    aux_io_param='NONE',
)

station.alazar.config(**ats_settings)

### ACQ controller settings

acq = avgctl

# acq.acq_time(0.1)
# acq.demod_frq(IF)



In [162]:
acq.update_acquisitionkwargs(**dict(
    samples_per_record=int(1e8 * 0.01)//128 * 128,
    records_per_buffer=1,
    buffers_per_acquisition=200,
    buffer_timeout = 10000,
    allocated_buffers = 100,
))

In [145]:
int(1e8 * 0.01)//128 * 128

999936

In [75]:
%%timeit -n 5
data = acq.acquisition()

1.1 s ± 1.36 ms per loop (mean ± std. dev. of 7 runs, 5 loops each)


In [163]:
data = acq.acquisition()
acq.data_dims(), acq.data_shape(), data.shape

(('records', 'samples', 'channels'), (1, 999936, 2), (1, 999936, 2))

In [147]:
for buffer in alazar.buffer_list:
    print(buffer._allocated)

True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True


In [ ]:
# plotdata = np.abs(np.squeeze(data))

# fig, ax = plt.subplots(1,1)
# ax.plot(acq.demod_tvals * 1e6, plotdata)

In [127]:
5 * 4  * 2e8 / (1024**2)

3814.697265625

In [50]:
t0 = time.perf_counter()
time.sleep(1)
print(time.perf_counter() - t0)

1.0006094688089888
